# Plant Disease Detection using MobileNetv4

- **Dataset** : New Plant Diseases Dataset
    - *Author of Dateset* : Samir Bhattarai
    - *Link* : [DataSet](https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset)

- **Architecture** : MobileNetv2

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras

2025-11-16 09:41:30.494772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763286090.574522     862 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763286090.599485     862 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-16 09:41:30.949939: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# --- CONSTANTS ---
IMG_SIZE = 224 # This is the standard input size for MobileNetV2
BATCH_SIZE = 32
NUM_CLASSES = 38
EPOCHS = 100 
TRAIN_DIR = "/mnt/d/College/DL/PlantDiseaseDetection-Project/New Plant Diseases Dataset(Augmented)/train"
VALID_DIR = "/mnt/d/College/DL/PlantDiseaseDetection-Project/New Plant Diseases Dataset(Augmented)/valid"


## Data Loading and Augumentation

In [3]:
# Load the training data
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    label_mode='int', # Your labels will be integers (0-37)
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Load the validation data
valid_ds = tf.keras.utils.image_dataset_from_directory(
    VALID_DIR,
    label_mode='int',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Get the class names for later
class_names = train_ds.class_names
print(f"Found {len(class_names)} classes: {class_names}...")

Found 70295 files belonging to 38 classes.


I0000 00:00:1763286168.399620     862 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 17572 files belonging to 38 classes.
Found 38 classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato

In [4]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
    # Note: Rescaling (1./255) will be the VERY first layer in our final model
], name="data_augmentation")

## PipeLine

In [ ]:
# Set AUTOTUNE to let TensorFlow find the best number of parallel calls
AUTOTUNE = tf.data.AUTOTUNE

# --- Create the full, optimized pipelines ---

def preprocess(image, label):
    # Apply augmentation to the batch
    image = data_augmentation(image)
    return image, label

train_ds = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache("my_train_cache").prefetch(buffer_size=AUTOTUNE) # <-- CHANGE THIS LINE

# Optimize the validation dataset
# This caches to a disk file named "my_valid_cache"
valid_ds = valid_ds.cache("my_valid_cache").prefetch(buffer_size=AUTOTUNE) # <-- CHANGE THIS LINE

In [6]:
from keras import layers, Model

# We use ReLU(6.0) as the activation, just like the original paper
# This helps with quantization (making the model work better on mobile)
relu6 = layers.ReLU(6.0)

def inverted_residual_block(
    x,               # Input tensor
    expansion_factor,  # How much to expand channels (e.g., 6)
    out_channels,    # Number of output channels
    strides,         # Stride for the depthwise conv (1 or 2)
):
    """
    Creates the MobileNetV2 Inverted Residual Block.
    """
    # Get the input channel count
    input_channels = x.shape[-1]
    
    # Calculate the number of channels for the "expansion" layer
    expanded_channels = input_channels * expansion_factor
    
    # Store the input for the residual connection
    shortcut = x

    # 1. --- Expansion (Pointwise) ---
    # Only expand if the expansion factor is not 1 (for the very first block)
    if expansion_factor != 1:
        x = layers.Conv2D(
            expanded_channels,
            (1, 1),
            padding='same',
            use_bias=False # No bias needed since we use Batch Norm
        )(x)
        x = layers.BatchNormalization()(x)
        x = relu6(x)

    # 2. --- Depthwise Convolution (Filtering) ---
    x = layers.DepthwiseConv2D(
        (3, 3),
        strides=strides,
        padding='same',
        use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)
    x = relu6(x)

    # 3. --- Projection (Pointwise, Linear Bottleneck) ---
    x = layers.Conv2D(
        out_channels,
        (1, 1),
        padding='same',
        use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)
    # --- Note: NO ACTIVATION HERE! This is the linear bottleneck. ---

    # 4. --- Residual Connection ---
    # We can only add the shortcut if:
    #   a) The stride is 1 (so spatial dimensions match)
    #   b) The input and output channels are the same
    if strides == 1 and input_channels == out_channels:
        x = layers.Add()([shortcut, x])
        
    return x

In [7]:
def create_mobilenet_v2(input_shape, num_classes):
    """
    Creates a full MobileNetV2 model from scratch.
    """
    inputs = layers.Input(shape=input_shape)

    # --- Rescaling Layer ---
    # This is a best practice. It scales [0-255] to [0-1].
    # It will run on the GPU as part of the model.
    x = layers.Rescaling(1./255)(inputs)
    
    # --- Stem (The first layer) ---
    x = layers.Conv2D(32, (3, 3), strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = relu6(x)

    # --- Body (The stack of inverted residual blocks) ---
    # We follow the table from the paper:
    # t = expansion_factor, c = out_channels, n = repeats, s = stride
    
    # t=1, c=16, n=1, s=1
    x = inverted_residual_block(x, 1, 16, 1)

    # t=6, c=24, n=2, s=2
    x = inverted_residual_block(x, 6, 24, 2)
    x = inverted_residual_block(x, 6, 24, 1)

    # t=6, c=32, n=3, s=2
    x = inverted_residual_block(x, 6, 32, 2)
    x = inverted_residual_block(x, 6, 32, 1)
    x = inverted_residual_block(x, 6, 32, 1)

    # t=6, c=64, n=4, s=2
    x = inverted_residual_block(x, 6, 64, 2)
    x = inverted_residual_block(x, 6, 64, 1)
    x = inverted_residual_block(x, 6, 64, 1)
    x = inverted_residual_block(x, 6, 64, 1)

    # t=6, c=96, n=3, s=1
    x = inverted_residual_block(x, 6, 96, 1)
    x = inverted_residual_block(x, 6, 96, 1)
    x = inverted_residual_block(x, 6, 96, 1)

    # t=6, c=160, n=3, s=2
    x = inverted_residual_block(x, 6, 160, 2)
    x = inverted_residual_block(x, 6, 160, 1)
    x = inverted_residual_block(x, 6, 160, 1)

    # t=6, c=320, n=1, s=1
    x = inverted_residual_block(x, 6, 320, 1)
    
    # --- Head (The final layers before classification) ---
    x = layers.Conv2D(1280, (1, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = relu6(x)
    
    # --- Classifier ---
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x) # Add dropout for regularization
    
    # Final output layer
    # We use 'softmax' for multi-class classification
    # We also specify dtype='float32' for numerical stability with mixed precision
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    # Create the model
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

## Model Creation

In [8]:
# Create the model
model = create_mobilenet_v2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3), 
    num_classes=NUM_CLASSES
)

# Print the model summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │        864 │ rescaling[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 7, 7,      │          0 │ batch_normalizat… │
│                     │ 1280)             │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d    │ (None, 112, 112,  │        288 │ re_lu[0][0]     

 Total params: 2,306,662 (8.80 MB)

 Trainable params: 2,272,550 (8.67 MB)

 Non-trainable params: 34,112 (133.25 KB)

## Compiling the Model with Mixed Precision for GPU Efficency

In [9]:
# Enable mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [10]:
# Create an Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

print("Model compiled successfully with mixed precision enabled.")

Model compiled successfully with mixed precision enabled.


In [11]:
# --- 1. Best Model Checkpoint ---
# Saves the complete model, only when validation accuracy improves.
best_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="/mnt/d/College/DL/PlantDiseaseDetection-Project/models/mobilenetv2_from_scratch.keras", # The final, best model
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# --- 2. Disaster Recovery Checkpoint ---
# Saves *only the weights* at the end of every single epoch.
# This is fast and ensures you can always resume from the last epoch.
latest_weights_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="/mnt/d/College/DL/PlantDiseaseDetection-Project/models/checkpoints/mobilenetv2_latest.weights.h5", # Note: .h5 is standard for weights
    monitor="val_loss",
    save_weights_only=True,  # This is the key difference
    save_best_only=False,    # We want the *latest*, not the best
    verbose=0                # No need to print every epoch
)

# --- 3. EarlyStopping ---
# This is the same as before.
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

# --- 4. Create the *new* list of callbacks ---
callbacks_list = [
    best_model_checkpoint,
    latest_weights_checkpoint,
    early_stopping_callback
]

# --- How to resume (IF it crashes) ---
# 1. Re-create your model (run the `create_mobilenet_v2` function)
# 2. Compile it (run the `model.compile()` cell)
# 3. Load the latest weights:
#    model.load_weights("mobilenetv2_latest.weights.h5")
# 4. Re-run `model.fit()` to continue training

In [12]:
from tqdm.keras import TqdmCallback # Make sure this is imported

final_callbacks_list = [
    best_model_checkpoint,
    latest_weights_checkpoint,
    early_stopping_callback,
    TqdmCallback(verbose=0) # The clean epoch-only progress bar
]

# ---  Start Training! ---
print("Starting model training...")
print(f"Training on {NUM_CLASSES} classes for {EPOCHS} max epochs.")
print("The TQDM progress bar will show epoch-by-epoch progress.")

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=EPOCHS,
    callbacks=final_callbacks_list,
    verbose=2  
)

print("-----------------------------------")
print("      TRAINING COMPLETE!           ")
print("-----------------------------------")

0epoch [00:00, ?epoch/s]

Starting model training...
Training on 38 classes for 100 max epochs.
The TQDM progress bar will show epoch-by-epoch progress.
Epoch 1/100


I0000 00:00:1763286196.913151    1083 service.cc:148] XLA service 0x757ad0005ae0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763286196.913225    1083 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-11-16 09:43:18.014982: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1763286200.103879    1083 cuda_dnn.cc:529] Loaded cuDNN version 90101
2025-11-16 09:44:01.721199: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng28{k2=1,k3=0} for conv (f32[32,96,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,16,112,112]{3,2,1,0}, f32[96,16,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"


Epoch 1: val_accuracy improved from None to 0.35488, saving model to /mnt/d/College/DL/PlantDiseaseDetection-Project/models/mobilenetv2_from_scratch.keras


2025-11-16 09:56:35.638939: W tensorflow/core/kernels/data/cache_dataset_ops.cc:332] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


2197/2197 - 817s - 372ms/step - accuracy: 0.6405 - loss: 1.1799 - val_accuracy: 0.3549 - val_loss: 5.1028
Epoch 2/100


2025-11-16 09:56:37.095025: W tensorflow/core/kernels/data/cache_dataset_ops.cc:332] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.



Epoch 2: val_accuracy improved from 0.35488 to 0.73185, saving model to /mnt/d/College/DL/PlantDiseaseDetection-Project/models/mobilenetv2_from_scratch.keras
2197/2197 - 460s - 209ms/step - accuracy: 0.8621 - loss: 0.4209 - val_accuracy: 0.7318 - val_loss: 1.0069
Epoch 3/100

Epoch 3: val_accuracy improved from 0.73185 to 0.83673, saving model to /mnt/d/College/DL/PlantDiseaseDetection-Project/models/mobilenetv2_from_scratch.keras
2197/2197 - 458s - 209ms/step - accuracy: 0.9116 - loss: 0.2687 - val_accuracy: 0.8367 - val_loss: 0.5799
Epoch 4/100

Epoch 4: val_accuracy did not improve from 0.83673
2197/2197 - 458s - 208ms/step - accuracy: 0.9353 - loss: 0.1942 - val_accuracy: 0.7412 - val_loss: 1.1294
Epoch 5/100

Epoch 5: val_accuracy improved from 0.83673 to 0.86000, saving model to /mnt/d/College/DL/PlantDiseaseDetection-Project/models/mobilenetv2_from_scratch.keras
2197/2197 - 449s - 204ms/step - accuracy: 0.9501 - loss: 0.1481 - val_accuracy: 0.8600 - val_loss: 0.4904
Epoch 6/100